In [1]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup 
from db_code.CWFAC_db import CWFACDB

In [2]:
# 1. Get the JSON data from the URL
url = "https://sleepy-poincare-71343e.netlify.app/tracking/records.json"
response = requests.get(url)
data = response.json()

# 2. Flatten the nested structure and convert to pandas DataFrame
records = []
for item in data:
    record = {
        "latitude": item["la"],
        "longitude": item["ln"],
        "date": item["d"]["date"],
        "collarId": item["d"]["collarId"],
        "serialId": item["d"]["serialId"],
        "positionId": item["d"]["positionId"]
    }
    records.append(record)

df = pd.DataFrame(records)

# Optional: preview the DataFrame
print(df)


       latitude  longitude                  date collarId   serialId  \
0     -2.276795  34.289950  2025-11-17T15:30:42Z    16653  T5HS-7304   
1     -1.459743  34.824200  2025-11-17T15:30:41Z    16655  T5HS-7306   
2     -2.289495  34.280628  2025-11-17T15:30:40Z    16652  T5HS-7303   
3     -2.289493  34.280533  2025-11-17T15:30:06Z    16652  T5HS-7303   
4     -2.276693  34.289920  2025-11-17T15:30:05Z    16653  T5HS-7304   
...         ...        ...                   ...      ...        ...   
11787 -1.913778  34.955347  2025-09-18T01:30:30Z    13288  T5HS-5583   
11788 -1.913387  34.955093  2025-09-18T01:30:01Z    13288  T5HS-5583   
11789 -1.913695  34.955328  2025-09-18T01:00:30Z    13288  T5HS-5583   
11790 -1.913497  34.954683  2025-09-18T00:30:30Z    13288  T5HS-5583   
11791 -1.913850  34.954728  2025-09-18T00:00:30Z    13288  T5HS-5583   

      positionId  
0      150675988  
1      150675957  
2      150675973  
3      150675958  
4      150675974  
...          ...  
11

In [3]:
# from datetime import datetime, timezone
# now = datetime.now(timezone.utc).isoformat()

In [4]:
# # df['serialId'].value_counts()
# len(df['serialId'].unique())
# # including collarId

In [5]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options

# # Setup headless Chrome
# options = Options()
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

# url = "https://www.serengeti-tracker.org/track/T5HS-5607"  # page with species
# driver.get(url)

# # Wait for the species element to load
# import time
# time.sleep(5)  # wait for JS to render content

# # Find the element containing species
# species_element = driver.find_element(By.CLASS_NAME, "details")
# print(species_element.text)  

# species_element.text

# driver.quit()


In [6]:
# species_df = pd.DataFrame(columns = ['serialId', 'species'])

In [7]:
# for serialId in df['serialId'].unique():
#     new_row_df = pd.DataFrame([{'serialId': serialId, 'species': 'zebra'}])
#     species_df = pd.concat([species_df, new_row_df], ignore_index = True)
# new_row_df = pd.DataFrame([{'serialId': 'testone', 'species': 'unknown'}])
# species_df = pd.concat([species_df, new_row_df], ignore_index = True)
# new_row_df = pd.DataFrame([{'serialId': 'testtwo', 'species': 'unknown'}])
# species_df = pd.concat([species_df, new_row_df], ignore_index = True)
# species_df

In [8]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options

# import re

# # Setup headless Chrome
# options = Options()
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

# temp_serialid = "T5HS-5570"
# species_df = pd.DataFrame(columns = ['serialId', 'species'])

# url = f"https://www.serengeti-tracker.org/track/{temp_serialid}"  # page with species

# try:
#     driver.get(url)

#     # Wait for the species element to load
#     import time
#     time.sleep(5)  # wait for JS to render content

#     # Find the element containing species
#     species_element = driver.find_element(By.CLASS_NAME, "details")
#     # print(species_element.text)  # should print 'zebra'
#     # print(type(species_element.text))
#     species = re.search(r"SPECIES\s+(.+?)\s+LAST TRACKED", species_element.text, re.DOTALL).group(1).strip()
#     print(species)
# except:
#     print(f"SOMETHING WENT WRONG WITH Serial ID {temp_serialid}")
#     species = "unknown"
# new_row_df = pd.DataFrame([{'serialId': temp_serialid, 'species': 'zebra'}])
# species_df = pd.concat([species_df, new_row_df], ignore_index = True)

# driver.quit()


In [9]:
# species_df

In [10]:
# for sp in set(list(species_df["species"].unique()) + ["unknown"]): 
#     # basically a convoluted way to get all the species including unknown always
#     print(sp)


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import re

# Setup headless Chrome
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

species_df = pd.DataFrame(columns = ['serialId', 'species'])

for serialId in df['serialId'].unique():

    url = f"https://www.serengeti-tracker.org/track/{serialId}"  # page with species
    try:
        driver.get(url)

        # Wait for the species element to load
        import time
        time.sleep(5)  # wait for JS to render content

        # Find the element containing species
        species_element = driver.find_element(By.CLASS_NAME, "details")
        # print(species_element.text)  # should print 'zebra'
        species = re.search(r"SPECIES\s+(.+?)\s+LAST TRACKED", species_element.text, re.DOTALL).group(1).strip()
        print(f"species of Serial ID {serialId} is {species}")
    except:
        print(f"SOMETHING WENT WRONG WITH Serial ID {serialId}, setting species name as 'unknown'")
        species = "unknown"

    new_row_df = pd.DataFrame([{'serialId': serialId, 'species': species}])
    species_df = pd.concat([species_df, new_row_df], ignore_index = True)
        
driver.quit()



species of Serial ID T5HS-7304 is Wildebeest
species of Serial ID T5HS-7306 is Wildebeest
species of Serial ID T5HS-7303 is Wildebeest
species of Serial ID T5HS-5583 is Wildebeest
species of Serial ID T5HS-6666 is Eland
species of Serial ID T5HS-5570 is Zebra
species of Serial ID T5HS-5594 is Wildebeest
species of Serial ID T5HS-7292 is Wildebeest
species of Serial ID T5HS-6142 is Zebra
species of Serial ID T5HS-7291 is Wildebeest
species of Serial ID T5HS-7301 is Wildebeest
species of Serial ID T5HS-7293 is Wildebeest
species of Serial ID T5HS-7300 is Wildebeest
SOMETHING WENT WRONG WITH Serial ID T5HS-7562, setting species name as 'unknown'
SOMETHING WENT WRONG WITH Serial ID T5HS-7564, setting species name as 'unknown'
species of Serial ID T5HS-7296 is Wildebeest
species of Serial ID T5HS-5608 is Wildebeest
species of Serial ID T5HS-7287 is Wildebeest
species of Serial ID T5HS-5601 is Wildebeest
SOMETHING WENT WRONG WITH Serial ID T5H-1861, setting species name as 'unknown'
SOMETHIN

In [12]:
merged_df = pd.merge(left = df, right = species_df, left_on = 'serialId', right_on = 'serialId')

In [13]:
merged_df["species"].unique()

array(['Wildebeest', 'Eland', 'Zebra', 'unknown'], dtype=object)

In [2]:
new_database = CWFACDB(path = "db_code/databasefile", create = False) # calling create = True on an existing thing will do nothing by default, to delete a database will require manual delection to prevent accidental deletion.

# new_database

In [3]:
sql = """
SELECT * FROM tObservations
"""

new_database.run_query(sql)

,serialId,date,collarId,latitude,longitude,positionId
0,T5HS-7304,2025-11-17T15:30:42Z,16653,-2.276795,34.289950,150675988
1,T5HS-7306,2025-11-17T15:30:41Z,16655,-1.459743,34.824200,150675957
2,T5HS-7303,2025-11-17T15:30:40Z,16652,-2.289495,34.280628,150675973
3,T5HS-7303,2025-11-17T15:30:06Z,16652,-2.289493,34.280533,150675958
4,T5HS-7304,2025-11-17T15:30:05Z,16653,-2.276693,34.289920,150675974
...,...,...,...,...,...,...
11786,T5HS-5583,2025-09-18T01:30:30Z,13288,-1.913778,34.955347,149589130
11787,T5HS-5583,2025-09-18T01:30:01Z,13288,-1.913387,34.955093,149589118
11788,T5HS-5583,2025-09-18T01:00:30Z,13288,-1.913695,34.955328,149589129
11789,T5HS-5583,2025-09-18T00:30:30Z,13288,-1.913497,34.954683,149589128


In [4]:
sql = """
SELECT * FROM tAnimal
"""

new_database.run_query(sql)

,serialId,species_id,first_scraped,last_scraped
0,T5HS-7304,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
1,T5HS-7306,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
2,T5HS-7303,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
3,T5HS-5583,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
4,T5HS-6666,2,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
5,T5HS-5570,1,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
6,T5HS-5594,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
7,T5HS-7292,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
8,T5HS-6142,1,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00
9,T5HS-7291,3,2025-11-17T20:24:02.655986+00:00,2025-11-17T21:29:06.427433+00:00


In [5]:
sql = """
SELECT * FROM tSpecies
"""

new_database.run_query(sql)

,species_id,species_name
0,1,Zebra
1,2,Eland
2,3,Wildebeest
3,4,unknown


In [6]:
# 1. Get the JSON data from the URL
url = "https://sleepy-poincare-71343e.netlify.app/tracking/records.json"
response = requests.get(url)
data = response.json()

# 2. Flatten the nested structure and convert to pandas DataFrame
records = []
for item in data:
    record = {
        "latitude": item["la"],
        "longitude": item["ln"],
        "date": item["d"]["date"],
        "collarId": item["d"]["collarId"],
        "serialId": item["d"]["serialId"],
        "positionId": item["d"]["positionId"]
    }
    records.append(record)

df = pd.DataFrame(records)

# Optional: preview the DataFrame
print(df)

       latitude  longitude                  date collarId   serialId  \
0     -2.271895  34.217840  2025-11-19T15:30:40Z    16642  T5HS-7293   
1     -2.010512  34.699328  2025-11-19T15:30:38Z    16641  T5HS-7292   
2     -2.271573  34.217813  2025-11-19T15:30:06Z    16642  T5HS-7293   
3     -2.010453  34.699093  2025-11-19T15:30:01Z    16641  T5HS-7292   
4     -2.306897  34.280237  2025-11-19T09:30:41Z    16650  T5HS-7301   
...         ...        ...                   ...      ...        ...   
11594 -1.955563  34.990795  2025-09-20T02:00:30Z    13288  T5HS-5583   
11595 -1.955487  34.990788  2025-09-20T01:30:30Z    13288  T5HS-5583   
11596 -1.955497  34.990782  2025-09-20T01:00:30Z    13288  T5HS-5583   
11597 -1.955417  34.990662  2025-09-20T00:30:30Z    13288  T5HS-5583   
11598 -1.953648  34.990277  2025-09-20T00:00:30Z    13288  T5HS-5583   

      positionId  
0      150707760  
1      150707774  
2      150707758  
3      150707763  
4      150705603  
...          ...  
11

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import re

# Setup headless Chrome
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

species_df = pd.DataFrame(columns = ['serialId', 'species'])

for serialId in df['serialId'].unique():

    url = f"https://www.serengeti-tracker.org/track/{serialId}"  # page with species
    try:
        driver.get(url)

        # Wait for the species element to load
        import time
        time.sleep(5)  # wait for JS to render content

        # Find the element containing species
        species_element = driver.find_element(By.CLASS_NAME, "details")
        # print(species_element.text)  # should print 'zebra'
        species = re.search(r"SPECIES\s+(.+?)\s+LAST TRACKED", species_element.text, re.DOTALL).group(1).strip()
        print(f"species of Serial ID {serialId} is {species}")
    except:
        print(f"SOMETHING WENT WRONG WITH Serial ID {serialId}, setting species name as 'unknown'")
        species = "unknown"

    new_row_df = pd.DataFrame([{'serialId': serialId, 'species': species}])
    species_df = pd.concat([species_df, new_row_df], ignore_index = True)
        
driver.quit()



species of Serial ID T5HS-7293 is Wildebeest
species of Serial ID T5HS-7292 is Wildebeest
species of Serial ID T5HS-7301 is Wildebeest
SOMETHING WENT WRONG WITH Serial ID T5HS-7564, setting species name as 'unknown'
species of Serial ID T5HS-7291 is Wildebeest
species of Serial ID T5HS-7300 is Wildebeest
species of Serial ID T5HS-7287 is Wildebeest
SOMETHING WENT WRONG WITH Serial ID T5HS-7562, setting species name as 'unknown'
species of Serial ID T5HS-7296 is Wildebeest
species of Serial ID T5HS-5608 is Wildebeest
species of Serial ID T5HS-6142 is Zebra
species of Serial ID T5HS-5594 is Wildebeest
species of Serial ID T5HS-5601 is Wildebeest
species of Serial ID T5HS-7298 is Wildebeest
SOMETHING WENT WRONG WITH Serial ID T5HS-7566, setting species name as 'unknown'
SOMETHING WENT WRONG WITH Serial ID T5HS-7561, setting species name as 'unknown'
species of Serial ID T5HS-5607 is Zebra
SOMETHING WENT WRONG WITH Serial ID T5H-1861, setting species name as 'unknown'
SOMETHING WENT WRONG 

In [8]:
merged_df = pd.merge(left = df, right = species_df, left_on = 'serialId', right_on = 'serialId')

In [9]:
new_database._load_data(df = merged_df)

loading data


In [10]:
sql = """
SELECT * FROM tObservations
"""

new_database.run_query(sql)

,serialId,date,collarId,latitude,longitude,positionId
0,T5HS-7304,2025-11-17T15:30:42Z,16653,-2.276795,34.289950,150675988
1,T5HS-7306,2025-11-17T15:30:41Z,16655,-1.459743,34.824200,150675957
2,T5HS-7303,2025-11-17T15:30:40Z,16652,-2.289495,34.280628,150675973
3,T5HS-7303,2025-11-17T15:30:06Z,16652,-2.289493,34.280533,150675958
4,T5HS-7304,2025-11-17T15:30:05Z,16653,-2.276693,34.289920,150675974
...,...,...,...,...,...,...
12011,T5HS-7298,2025-11-15T21:30:33Z,16647,-2.293332,34.332863,150694312
12012,T5HS-5607,2025-11-15T15:30:41Z,13312,-2.205385,34.705243,150689790
12013,T5HS-7302,2025-11-15T15:30:30Z,16651,-2.145550,34.855872,150678415
12014,T5HS-7302,2025-11-15T09:30:42Z,16651,-2.148665,34.886712,150678414


In [11]:
sql = """
SELECT * FROM tAnimal
"""

new_database.run_query(sql)

,serialId,species_id,first_scraped,last_scraped
0,T5HS-7304,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
1,T5HS-7306,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
2,T5HS-7303,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
3,T5HS-5583,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
4,T5HS-6666,2,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
5,T5HS-5570,1,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
6,T5HS-5594,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
7,T5HS-7292,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
8,T5HS-6142,1,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00
9,T5HS-7291,3,2025-11-17T20:24:02.655986+00:00,2025-11-19T18:06:51.534819+00:00


In [12]:
sql = """
SELECT * FROM tObservations 
WHERE serialID = 'T5HS-7303'
"""

new_database.run_query(sql)

,serialId,date,collarId,latitude,longitude,positionId
0,T5HS-7303,2025-09-18T03:30:41Z,16652,-2.169220,34.439965,149606364
1,T5HS-7303,2025-09-18T07:30:30Z,16652,-2.200257,34.451910,149606365
2,T5HS-7303,2025-09-18T11:30:30Z,16652,-2.202940,34.450767,149606366
3,T5HS-7303,2025-09-18T15:30:30Z,16652,-2.204747,34.450297,149606367
4,T5HS-7303,2025-09-18T19:30:30Z,16652,-2.203817,34.451582,149606368
...,...,...,...,...,...,...
362,T5HS-7303,2025-11-16T21:30:42Z,16652,-2.291437,34.308123,150675970
363,T5HS-7303,2025-11-17T03:30:30Z,16652,-2.291312,34.300992,150675971
364,T5HS-7303,2025-11-17T09:30:30Z,16652,-2.298707,34.287472,150675972
365,T5HS-7303,2025-11-17T15:30:06Z,16652,-2.289493,34.280533,150675958


In [23]:
sql = """
SELECT tObservations.*, tAnimal.species_id, tAnimal.first_scraped, tAnimal.last_scraped, tSpecies.species_name
FROM tObservations
JOIN tAnimal ON tObservations.serialId = tAnimal.serialId
JOIN tSpecies ON tAnimal.species_id = tSpecies.species_id
"""

example_full_data = new_database.run_query(sql)

example_full_data.to_csv("example_full_data.csv")